In [1]:
import alphaspace2 as al
import mdtraj
import numpy as np
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import fcluster, linkage
from alphaspace2.functions import _binCluster, _group
from alphaspace2.Cluster import _DPocket
import scipy
import pandas as pd

In [2]:
from communities import genCommunityPocket,CoreCluster
import features

## In this tutorial, we will be community detection for the Fragment to Lead analysis of the HSP90 structures

#### We apply the ensemble pocket approach to get the communities associated with fragment and their corresponding lead structures
#### In the first part we look at the communities and core pockets of a bound fragment structure for HSP90
#### In the second part, we look at the different metrics that show how lead structures have better complementarity to the binding site compared to their fragment counterparts

#### First strip hydrogens from pdbqt

In [3]:
def strip_h(input_file,output_file):
    '''
    input_file and output_file need to be in pdb or pdbqt format 
    '''
    inputlines = open(input_file,'r').readlines()
    no_h_lines = [l for l in inputlines if not l.split()[-1].startswith('H')] 
    output = open(output_file,'w')
    output.writelines(no_h_lines)
    output.close()

In [4]:
strip_h("Fragment_to_Lead/protein_2wi2.pdbqt","Fragment_to_Lead/protein_2wi2_noH.pdbqt")
strip_h("Fragment_to_Lead/protein_2wi7.pdbqt","Fragment_to_Lead/protein_2wi7_noH.pdbqt")

In [5]:
hsp90_frag_prot = mdtraj.load('Fragment_to_Lead/protein_2wi2.pdb')
al.annotateVinaAtomTypes(pdbqt="Fragment_to_Lead/protein_2wi2_noH.pdbqt", receptor=hsp90_frag_prot)
hsp90_lead_prot = mdtraj.load('Fragment_to_Lead/protein_2wi7.pdb')
al.annotateVinaAtomTypes(pdbqt="Fragment_to_Lead/protein_2wi7_noH.pdbqt", receptor=hsp90_lead_prot)

In [6]:
hsp90_frag_ss, hsp90_lead_ss = al.Snapshot(), al.Snapshot()
hsp90_frag_ss.run(hsp90_frag_prot)
hsp90_lead_ss.run(hsp90_lead_prot)
frag_lead = al.Trajectory(snapshots=[hsp90_frag_ss,hsp90_lead_ss])
frag_lead.gen_dpockets(clust_distance=4.7)

### For the next cell, we store pocket properties into dictionaries which will be used for generating the surface communities

In [7]:
temp_space_dict = {'Frag':{},'Lead':{}}
temp_coords_dict = {'Frag':{},'Lead':{}}
temp_contact_dict = {'Frag':{},'Lead':{}}
temp_score_dict = {'Frag':{},'Lead':{}}
temp_centroids_dict = {'Frag':{},'Lead':{}}
for dpx,dp in enumerate(frag_lead.dpockets):
    pockets = list(dp.pockets)
    if pockets[0].score != 0:
        pock = pockets[0]
        temp_coords_dict['Frag'][dpx] = [list(b.xyz) for b in pock.betas]
        temp_score_dict['Frag'][dpx] = np.array([min(b.scores) for b in  pock.betas])
        temp_space_dict['Frag'][dpx] = np.array([b.space for b in  pock.betas])
        temp_contact_dict['Frag'][dpx] = list(pock.lining_atoms_idx)
        temp_centroids_dict['Frag'][dpx] = pock.centroid
    if pockets[1].score != 0:
        pock = pockets[1]
        temp_coords_dict['Lead'][dpx] = [list(b.xyz) for b in pock.betas]
        temp_score_dict['Lead'][dpx] = np.array([min(b.scores) for b in  pock.betas])
        temp_space_dict['Lead'][dpx] = np.array([b.space for b in  pock.betas])
        temp_contact_dict['Lead'][dpx] = list(pock.lining_atoms_idx)
        temp_centroids_dict['Lead'][dpx] = pock.centroid

In [8]:
prot_frag_coords = hsp90_frag_prot.xyz[0]*10
frag_surface_communities = genCommunityPocket(prot_frag_coords, temp_centroids_dict['Frag'], temp_space_dict['Frag'], \
                                            temp_contact_dict['Frag'], temp_score_dict['Frag'], \
                                            corecut = 100, auxcut = 30, tight_option = True, tight_cutoff_core = 12.5, tight_cutoff_aux = 6.5)

### Loading ligands for detecting binding site pockets

In [9]:
ligand_frag = mdtraj.load('Fragment_to_Lead/ligand_frag_2wi2.pdb')
ligand_lead = mdtraj.load('Fragment_to_Lead/ligand_lead_2wi7.pdb')

### Here we get the list of communities that overlap with the fragment ligand 
### In this case, there is only one overlapping community which is the top-ranking community by space (which has index 0) 

In [10]:
binding_communities = []
for cx,community in frag_surface_communities.items():
    temp_binding_community_coords = []
    for pock in community['core_pockets'] + community['aux_pockets']:
        temp_binding_community_coords.extend(temp_coords_dict['Frag'][pock])
    dist=scipy.spatial.distance.cdist(temp_binding_community_coords,ligand_frag.xyz[0]*10)
    bool_arr = np.any(dist<1.8,axis = 1)    
    if np.any(bool_arr):
        binding_communities.append(cx)

### Here we get local ranking and properties of high-scoring pockets in the binding site community and save them into a dataframe

In [11]:
pocket_data = pd.DataFrame(columns=('pocket_ix', 'score', '%optimized','space','%space_occupied'))

In [13]:
for cx in binding_communities:
    community = frag_surface_communities[cx]
    for pock in community['core_pockets'] + community['aux_pockets']:
        pocket_coord = np.array(temp_coords_dict['Frag'][pock])
        pocket_space = np.array(temp_space_dict['Frag'][pock])
        pocket_score = np.array(temp_score_dict['Frag'][pock])
        dist=scipy.spatial.distance.cdist(pocket_coord,ligand_frag.xyz[0]*10)
        bool_arr = np.any(dist<1.8,axis = 1)    
        optimized = 100*np.sum(pocket_score[bool_arr])/np.sum(pocket_score)
        space_occupied = 100*np.sum(pocket_space[bool_arr])/np.sum(pocket_space)
        temp_row = [pock,np.sum(pocket_score),optimized,np.sum(pocket_space),space_occupied]
        pocket_data = pocket_data.append(pd.Series(temp_row,index=pocket_data.columns),ignore_index=True)

In [14]:
pocket_data

,pocket_ix,score,%optimized,space,%space_occupied
0,6.0,-2.325968,-0.000000,205.921570,0.000000
1,141.0,-3.659364,62.386755,203.603363,70.840921
2,142.0,-4.234781,29.320790,145.997589,34.714360
3,143.0,-5.977964,68.868089,446.135773,50.351348
4,144.0,-1.198709,-0.000000,36.762062,0.000000
5,146.0,-2.382712,-0.000000,99.947273,0.000000
6,6.0,-2.325968,-0.000000,205.921570,0.000000
7,141.0,-3.659364,62.386755,203.603363,70.840921
8,142.0,-4.234781,29.320790,145.997589,34.714360
9,143.0,-5.977964,68.868089,446.135773,50.351348


#### We see that pocket 143, 142, and 141 are the top 3 ranking pockets and are also the most optimized/occupied
#### Pocket 6 and 146 are not occupied by the fragment and would be a good starting point for ligand optimization

### In this section we take a look at pocket/ligand metrics to evaluate the complementarity between the pocket and ligands in the lead structure

In [14]:
prot_lead_coords = hsp90_lead_prot.xyz[0]*10

In [15]:
lead_surface_communities = genCommunityPocket(prot_lead_coords, temp_centroids_dict['Lead'], temp_space_dict['Lead'], \
                                            temp_contact_dict['Lead'], temp_score_dict['Lead'], \
                                            corecut = 100, auxcut = 30, tight_option = True, tight_cutoff_core = 12.5, tight_cutoff_aux = 6.5)

In [16]:
binding_communities = []
for cx,community in lead_surface_communities.items():
    temp_binding_community_coords = []
    for pock in community['core_pockets'] + community['aux_pockets']:
        temp_binding_community_coords.extend(temp_coords_dict['Lead'][pock])
    dist=scipy.spatial.distance.cdist(temp_binding_community_coords,ligand_lead.xyz[0]*10)
    bool_arr = np.any(dist<1.8,axis = 1)    
    if np.any(bool_arr):
        binding_communities.append(cx)

### Like in the fragment structure, the highest ranking community is also the only detected binding site for the lead molecule is community index 0

In [35]:
binding_communities

[0]

In [17]:
pock_set = set()
for cx in binding_communities:
    for pock in lead_surface_communities[cx]['core_pockets'] + lead_surface_communities[cx]['aux_pockets']:
        pock_set.add(pock)

### We save the pocket coordinates, score, and space into lists which we will use to measure the complimentarity of the lead vs the fragment ligand using %Optimized, % space occupied, and % volume overlap scores

In [19]:
lead_binding_community_coords = []
lead_binding_community_space = []
lead_binding_community_score = []
for pock in pock_set:
    lead_binding_community_coords.extend(temp_coords_dict['Lead'][pock])
    lead_binding_community_space.extend(temp_space_dict['Lead'][pock])
    lead_binding_community_score.extend(temp_score_dict['Lead'][pock])
lead_binding_community_coords = np.array(lead_binding_community_coords)
lead_binding_community_space = np.array(lead_binding_community_space)
lead_binding_community_score = np.array(lead_binding_community_score)

In [20]:
bool_arr = np.any(dist<=1.6,axis = 1)

### The Lead structure improves the various metrics in the Lead binding site community compared to the Fragment Structure by >2X. Maximizing these metrics by more elaborated lead structures can be used to guide the design of more potent inhibitors

In [21]:
for lx,lig_coords in [('Frag',ligand_frag.xyz[0]*10),('Lead',ligand_lead.xyz[0]*10)]:
    dist = scipy.spatial.distance.cdist(lead_binding_community_coords,lig_coords)
    bool_arr = np.any(dist<=1.6,axis = 1)
    optimized = round(100*np.sum(lead_binding_community_score[bool_arr])/np.sum(lead_binding_community_score),2)
    space_occpupied = round(100*np.sum(lead_binding_community_space[bool_arr])/np.sum(lead_binding_community_space),2)
    overlap_volume = features._get_overlap_volume(lead_binding_community_coords,lig_coords)
    total_volume = features._get_grid_volume(lead_binding_community_coords)
    overlap = round(100*overlap_volume/total_volume,2)
    print(lx, '%Optimized : ' +str(optimized)+ ', %Space Occupied : '+str(space_occpupied) \
         ,', %Overlap Volume : ' + str(overlap))

Frag %Optimized : 13.74, %Space Occupied : 16.19 , %Overlap Volume : 10.55
Lead %Optimized : 36.88, %Space Occupied : 49.19 , %Overlap Volume : 27.28


### In this section, we present some scripts that can be use to visualize the Lead communities along with the corresponding pictures. Here we write different attribute files to help visualize pocket communities. All files will be written in the Fragment_to_Lead/pocket_communities file

#### Attribute files allows us to visualize different features as attributes pirojected to the beta clusters or protein surface in Chimera

In [22]:
attribute_file = ["attribute: beta_score","recipient: atoms"]
ixx = 0
for pock in temp_score_dict['Lead']:
    for fx,beta_score in enumerate(temp_score_dict['Lead'][pock]):
        beta_score = round(beta_score,2)
        attribute_file.append('\t' + ':'+str(pock)+' & :BAC & @/serialNumber='+str(fx)+'\t'+str(beta_score))
with open('Fragment_to_Lead/pocket_communities/beta_score_attributes.txt','w') as f:
    f.write('\n'.join(attribute_file))

In [23]:
pocket_score_attribute_file = ["attribute: pocket_score","recipient: atoms"]
pocket_optimize_attribute_file = ["attribute: optimize","recipient: atoms"]
for pock in temp_score_dict['Lead']:
    coords = temp_coords_dict['Lead'][pock]
    score = temp_score_dict['Lead'][pock]
    dist = scipy.spatial.distance.cdist(coords,ligand_lead.xyz[0]*10)
    bool_arr = np.any(dist<=1.6,axis = 1)
    optimized = round(100*np.sum(score[bool_arr])/np.sum(score),2)
    pocket_score = round(np.sum(score),2)
    pocket_score_attribute_file.append('\t' + ':'+str(pock)+' & @BAO \t'+str(pocket_score))
    pocket_optimize_attribute_file.append('\t' + ':'+str(pock)+' & @BAO \t'+str(optimized))
with open('Fragment_to_Lead/pocket_communities/pocket_score_attributes.txt','w') as f:
    f.write('\n'.join(pocket_score_attribute_file))
with open('Fragment_to_Lead/pocket_communities/pocket_optimize_attributes.txt','w') as f:
    f.write('\n'.join(pocket_optimize_attribute_file))

In [24]:
for sx,data in lead_surface_communities.items():
    community_pdb = []
    lining_atoms = set()
    for pock in data['core_pockets'] + data['aux_pockets']:
        score = np.sum(temp_score_dict['Lead'][pock])
        centroid = temp_centroids_dict['Lead'][pock]
        lining_atoms.update(list(temp_contact_dict['Lead'][pock]))
        coords = temp_coords_dict['Lead'][pock]
        dist=scipy.spatial.distance.cdist(coords,ligand_lead.xyz[0]*10)
        bool_arr = np.any(dist<1.8,axis = 1)    
        opt = np.sum(temp_score_dict['Lead'][pock][bool_arr])/score
        if score <= -2.5:
            res = 'BHI'
        elif score > -2.5 and score <= -1.5:
            res = 'BMI'
        elif score > -1.5:
            res = 'BLI'
        community_pdb.append('ATOM  '+str(1).rjust(5)+'  '+'BAO'+' '+res+'   '+str(pock).rjust(3)+'    '+str(round(centroid[0],3)).rjust(8)+str(round(centroid[1],3)).rjust(8)+str(round(centroid[2],3)).rjust(8)+str(0.0).rjust(6)+str(opt).rjust(6)+'           C')
        for fx,ff in enumerate(zip(bool_arr,coords)):
            bx,coord = ff
            beta_score = round(temp_score_dict['Lead'][pock][fx],2)
            if bx:
                community_pdb.append('ATOM  '+str(fx).rjust(5)+'  '+'CON'+' '+'BAC'+'   '+str(pock).rjust(3)+'    '+str(round(coord[0],3)).rjust(8)+str(round(coord[1],3)).rjust(8)+str(round(coord[2],3)).rjust(8)+str(beta_score).rjust(6)+str(0.00).rjust(6)+'           C')
            else:
                community_pdb.append('ATOM  '+str(fx).rjust(5)+'  '+'NON'+' '+'BAC'+'   '+str(pock).rjust(3)+'    '+str(round(coord[0],3)).rjust(8)+str(round(coord[1],3)).rjust(8)+str(round(coord[2],3)).rjust(8)+str(beta_score).rjust(6)+str(0.00).rjust(6)+'           C')
        community_pdb.append('TER')
    with open('Fragment_to_Lead/pocket_communities/community_'+str(sx)+'.pdb','w') as f:
        f.write('\n'.join(community_pdb))
            

In [25]:
protein_topology = []
for resatom in hsp90_lead_prot.top.atoms:
    res,atom_index,res_index,atom,element = resatom.residue.name, resatom.index, resatom.residue.index, resatom.name, resatom.element.symbol
    protein_topology.append([res,atom_index,res_index,atom,element])
protein_topology = np.array(protein_topology)

In [26]:
with open('Fragment_to_Lead/pocket_communities/protein.pdb','w') as f:
    for top,coord in zip(protein_topology,prot_lead_coords):    
        res,atom_index,res_index,atom,element = top 
        f.write('ATOM  '+str(atom_index).rjust(5)+'  '+atom.ljust(3)+' '+res+'   '+str(res_index).rjust(3)+'    '+str(round(coord[0],3)).rjust(8)+str(round(coord[1],3)).rjust(8)+str(round(coord[2],3)).rjust(8)+str(0.0).rjust(6)+str(0.00).rjust(6)+'           '+element+'\n')
        

In [27]:
ligand_topology = []
for resatom in ligand_lead.top.atoms:
    res,atom_index,res_index,atom,element = resatom.residue.name, resatom.index, resatom.residue.index, resatom.name, resatom.element.symbol
    ligand_topology.append([res,atom_index,res_index,atom,element])
ligand_topology = np.array(ligand_topology)

with open('Fragment_to_Lead/pocket_communities/ligand.pdb','w') as f:
    for top,coord in zip(ligand_topology,ligand_lead.xyz[0]*10):    
        res,atom_index,res_index,atom,element = top 
        f.write('HETATM'+str(atom_index).rjust(5)+'  '+atom.ljust(3)+' '+res+'   '+str(res_index).rjust(3)+'    '+str(round(coord[0],3)).rjust(8)+str(round(coord[1],3)).rjust(8)+str(round(coord[2],3)).rjust(8)+str(0.0).rjust(6)+str(0.00).rjust(6)+'           '+element+'\n')
        

### This is the image what we should expect from opening the AS_viz_template.py and loading the first saved scene. High-, mid-, and low-scoring pockets from the binding site community is colored green, blue, and rosybrown respectively. This view allows users to immediately visualize high and mid-scoring pockets that can be exploited for fragment elaboration

![title](Fragment_to_Lead/pocket_communities/Fragment_to_lead_community_1.png)

### This is the image what we should expect from opening the AS_viz_template.py and loading the second saved scene. Pockets centroids are colored by their %optimized. Highly optimized pockets are colored green while completly unoccupied pockets are colored yellow. Pockets with intermediate %optimized scores are colored in between. This view allows users to identify contact pockets near the ligand which can be sites of fragment elaboration or partially occupied pockets that should be the focus for ligand design

![title](Fragment_to_Lead/pocket_communities/Fragment_to_lead_community_2.png)

### This is the image what we should expect from opening the AS_viz_template.py and loading the third saved scene. In this view, beta atoms are colored based on the vina score corresponding to their most favorable probe atoms. The most favorable beta atoms are colored green while beta low-scoring beta atoms are colored orange. This view allows a more fine-grained view of favorable sites on the surface

![title](Fragment_to_Lead/pocket_communities/Fragment_to_lead_community_3.png)